In [1]:
import json
import pandas as pd
import gzip
import numpy as np
from sklearn.externals import joblib
from github import Github
import gensim
from langdetect import detect
from collections import Counter

In [2]:
# load toy example
# File from: wget http://data.gharchive.org/2018-07-16-15.json.gz

events = []
for hour in range(15, 16):
    with gzip.open('/Users/amy/git/MetaGithub/data/2018-07-16-%d.json.gz' %hour) as f:
        for line in f:
            events.append(json.loads(line.strip()))

In [3]:
# get all watch events
# The WatchEvent corresponds to starring a repository, not watching. 
starred = [event for event in events if event['type'] == 'WatchEvent']

# count how many times each repo was starred during this time
# maps repo name to count of star events
repo2count = {}

for event in starred:
    
    repo_name = event['repo']['name']
    
    if repo_name not in repo2count:
        repo2count[repo_name] = 1
    else:
        repo2count[repo_name] += 1


In [4]:
# record the names of repos that were starred more than k times during this time
# i.e. get popular repos during this time period
# we can change this code to find the top xx repos instead, etc.

popular_repos = []
k = 10 # this was chosen arbitrarily 

for repo_name in repo2count:
    if repo2count[repo_name] > k:
        popular_repos.append(repo_name)
        
len(popular_repos)

55

In [6]:
# github object using my personal access token
g = Github('dc346ff7262b4b55dcc8ea6abe35c9c3170d415b')

readme_files = []
langs = [] # language of readmes

# for each popular repo
for repo_name in popular_repos:
    
    # get the repository
    repo = g.get_repo(repo_name)
    
    # get the readme file (Shaun's code)
    try:
        readMe = repo.get_contents('README.md')
        readMe = readMe.decoded_content
        readMe = readMe.decode("utf-8")
        readme_files.append(readMe)
        lang = detect(readMe)
        langs.append(lang)
    except:
        readMe ="No README.md found."
        readme_files.append(readMe)
        lang = detect(readMe)
        langs.append(lang)




In [10]:
# check length 
len(popular_repos) == len(readme_files)

True